In [2]:
import os
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import DirectoryLoader
# from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

In [3]:
loader = DirectoryLoader("/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data",
                    glob="**/*.txt",
                    loader_cls=TextLoader,
                    show_progress=True,
                    loader_kwargs={"encoding":"utf-8"})

data = loader.load()

100%|██████████| 1834/1834 [00:00<00:00, 20210.33it/s]


In [4]:
data[0]

Document(page_content=' \n  Автор  ООО "НПЦ "1С"  Концепция  миграции данных Инструкция \nпользователей «D -1C1-1.10.39 Сверка \nвзаиморасчетов с контрагентами»  Страница  14  \nСтраниц  52  \nВерсия   5  \n  \n Инструкция пользователей D -1C1 -1.10.39  \nСверка взаиморасчетов с контрагентами  \n  \n  \n   \n \nРисунок 5: Вкладка «Основное» документа «Акт инвентаризации расчетов»   \nНа вкладке «Счета » устанавливаются счета расчета, по которым необходимо \nпровести инвентаризацию. По кнопке « Настроить » можно выполнить настройку счетов \n(Рисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов» ).  \n \nРисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов»   \nПри нажатии на кнопку « Перезаполнить » происходит заполнение табличных частей \nна вкладке « Счета » и « Задолженность » (Рисунок 7: Заполненная вкладка «Счета» ).  \n', metadata={'source': '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data/Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов_с_контрагент

In [65]:
text_splitter=RecursiveCharacterTextSplitter( # TODO: semantic chunking
    chunk_size=1000, # TODO: tune this
    chunk_overlap=70
)

chunks = text_splitter.split_documents(data)

In [66]:
chunks[0]

Document(page_content='Автор  ООО "НПЦ "1С"  Концепция  миграции данных Инструкция \nпользователей «D -1C1-1.10.39 Сверка \nвзаиморасчетов с контрагентами»  Страница  14  \nСтраниц  52  \nВерсия   5  \n  \n Инструкция пользователей D -1C1 -1.10.39  \nСверка взаиморасчетов с контрагентами  \n  \n  \n   \n \nРисунок 5: Вкладка «Основное» документа «Акт инвентаризации расчетов»   \nНа вкладке «Счета » устанавливаются счета расчета, по которым необходимо \nпровести инвентаризацию. По кнопке « Настроить » можно выполнить настройку счетов \n(Рисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов» ).  \n \nРисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов»   \nПри нажатии на кнопку « Перезаполнить » происходит заполнение табличных частей \nна вкладке « Счета » и « Задолженность » (Рисунок 7: Заполненная вкладка «Счета» ).', metadata={'source': '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data/Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов_с_контрагентами/page_

In [67]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
db = FAISS.from_documents(chunks, embeddings)

In [68]:
dir_name = 'unstructured_vdb'
all_path_dir = os.path.join("..", "data", dir_name)
if not os.path.exists(all_path_dir):
    os.makedirs(all_path_dir)

In [69]:
faiss_vdb = "index"

In [70]:
db.save_local(all_path_dir, faiss_vdb)

### Тестирование

In [71]:
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

In [72]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db2 = FAISS.load_local(
    os.path.join("..", "data", 'unstructured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [73]:
query = "Что является целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО?"
docs = db2.similarity_search(query)

In [74]:
docs

[Document(page_content='Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: \nЦифровой Росатом с ЕОСДО » Страница 10 \nСтраниц 35 \nВерсия  6 \n1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из \nЕОСДО» \nСхематично интеграционный сценарий изображён на Рис унке 1.2: \nРисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» \nФормирование \nреестра и отправка \nреестра в ЕОСДО \nПолучение и \nобработка реестра Выгрузка скан-\nобразов \nдокументов на \nфайловый сервер Подтверждение \nзавершения \nвыгрузки \nM .O.3 M .I.3\n \nЦелью интеграционного сценария «Массовая выгрузка д окументов из \nЕОСДО» является автоматизация предоставления реестр а скан-образов \nдокументов в контролирующие органы. \nЗависимые бизнес-процессы:  \n\uf0b7 D-1C1-1.10.19 Формирование налоговой декларации по НП; \n\uf0b7 D-1C1-1.10.15 Формирование налоговой декларации по НДС; \n\uf0b7 D-1C1-1.10.35 Учет средств целевого финансирования;  \n\uf0b7 D-1C1-1.10.40 Формирование бухгал

In [76]:
print(docs[0].page_content)
page = docs[0].metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
file_name = docs[0].metadata['source'].split('/')[-2] + ".pdf"
print(f"Страница: {page}", f"Исходный документ: {file_name}")

Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: 
Цифровой Росатом с ЕОСДО » Страница 10 
Страниц 35 
Версия  6 
1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из 
ЕОСДО» 
Схематично интеграционный сценарий изображён на Рис унке 1.2: 
Рисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» 
Формирование 
реестра и отправка 
реестра в ЕОСДО 
Получение и 
обработка реестра Выгрузка скан-
образов 
документов на 
файловый сервер Подтверждение 
завершения 
выгрузки 
M .O.3 M .I.3
 
Целью интеграционного сценария «Массовая выгрузка д окументов из 
ЕОСДО» является автоматизация предоставления реестр а скан-образов 
документов в контролирующие органы. 
Зависимые бизнес-процессы:  
 D-1C1-1.10.19 Формирование налоговой декларации по НП; 
 D-1C1-1.10.15 Формирование налоговой декларации по НДС; 
 D-1C1-1.10.35 Учет средств целевого финансирования;  
 D-1C1-1.10.40 Формирование бухгалтерской отчетности .
Страница: 10 Исходный документ: Инструкция Интегра